In [30]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import fits
import os
import glob

In [31]:
# Define the file path (assuming it's in the same directory)
file_name = "final_all_hxmt_lc_periods_chisq_LE.txt"

# Read the period files for each instrument separately
periods_le_df = pd.read_csv("final_all_hxmt_lc_periods_chisq_LE.txt", sep='\s+', comment="#")
periods_me_df = pd.read_csv("final_all_hxmt_lc_periods_chisq_ME.txt", sep='\s+', comment="#")
periods_he_df = pd.read_csv("final_all_hxmt_lc_periods_chisq_HE.txt", sep='\s+', comment="#")

def extract_exposure_id_from_path(lc_path):
    """
    Extract the exposure ID from LC file path by parsing filename.
    
    Parameters:
        lc_path (str): Full path to lightcurve file.
        
    Returns:
        str: Extracted exposure ID (e.g., P050427900101).
    """
    filename = os.path.basename(lc_path)
    exp_id = filename.split('_')[0]  # Split at first underscore
    return exp_id

# Apply it to the whole dataframe
# Add Exposure_ID column
for df in [periods_le_df, periods_me_df, periods_he_df]:
    df["Exposure_ID"] = df["LC_File_Path"].apply(extract_exposure_id_from_path)

# Check
print(periods_le_df.head())

                                        LC_File_Path   Period  Chi-square  \
0  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79482     10210.0   
1  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79473      4372.0   
2  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79434      4448.0   
3  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79438      5297.0   
4  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79446     15670.0   

    Date        freq      PErr    Exposure_ID  
0  20154  102.092905  0.000022  P050419600101  
1  20154  102.092905  0.000072  P050419600102  
2  20154  102.103329  0.000318  P050419600103  
3  20155  102.103329  0.000070  P050419600201  
4  20155  102.092905  0.000025  P050419600202  


In [32]:
# Base directory for traversal
base_dir = "/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline"

# Find all .lc files under pipeline_output folders
all_lc_files = sorted(glob.glob(os.path.join(base_dir, "**", "pipeline_output", "*lcnet_*.lc"), recursive=True))

print(f"Total LC files found: {len(all_lc_files)}")
print("Example files:")
print(all_lc_files[:5])  # print a few for sanity check


# Helper function to extract exposure ID and instrument
def extract_info_from_lc(filepath):
    """
    Extracts the exposure ID and instrument from HXMT LC file path.
    
    Example filename: 'P050419600101_HE_lcnet_27-250keV_0.05s_all.lc'
    Returns: ('P050419600101', 'HE')
    """
    filename = os.path.basename(filepath)
    parts = filename.split('_')
    exposure_id = parts[0]  # First part before first '_'
    instrument = parts[1]   # Second part is instrument (HE, ME, LE)
    return exposure_id, instrument


# Prepare separate lists for LE, ME, HE
le_files = []
me_files = []
he_files = []

exposure_ids_le = []
exposure_ids_me = []
exposure_ids_he = []

# Classify
for f in all_lc_files:
    exposure_id, instrument = extract_info_from_lc(f)
    
    if instrument == "LE":
        le_files.append(f)
        exposure_ids_le.append(exposure_id)
    elif instrument == "ME":
        me_files.append(f)
        exposure_ids_me.append(exposure_id)
    elif instrument == "HE":
        he_files.append(f)
        exposure_ids_he.append(exposure_id)
    else:
        print(f"Warning: Unknown instrument in file {f}")


# Summarize
print(f"\nFound {len(le_files)} LE files")
print(f"Found {len(me_files)} ME files")
print(f"Found {len(he_files)} HE files")

print("\nExample LE files:")
print(le_files[:3])
print("\nExample ME files:")
print(me_files[:3])
print("\nExample HE files:")
print(he_files[:3])

print("\nExample LE exposure IDs:")
print(exposure_ids_le[:3])


Total LC files found: 164
Example files:
['/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600101_HE_lcnet_27-250keV_0.05s_all.lc', '/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600101_LE_lcnet_2-10keV_0.05s.lc', '/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600101_ME_lcnet_10-35keV_0.05s.lc', '/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600102_HE_lcnet_27-250keV_0.05s_all.lc', '/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600102_LE_lcnet_2-10keV_0.05s.lc']

Found 52 LE files
Found 59 ME files
Found 53 HE files

Example LE files:
['/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P05041

In [35]:
# Settings
nphase = 64
epoch = '20112.43540056619'
output_base_folder = "./hxmt_efold_output"
os.makedirs(output_base_folder, exist_ok=True)  # Create if not exists


# Which instruments to process
process_LE = True
process_ME = True
process_HE = True
# Helper: Run efold in output directory
def run_efold(lc_file, period, output_file, output_dir):
    short_name = os.path.basename(output_file)  # Just filename, no path

    cmd = (
        f"echo '-' | efold 1 {lc_file} "
        f"outfile={short_name} "
        f"dper={period} "
        f"nphase={nphase} "
        f"epochfo=1 "
        f"sepoch={epoch} "
        f"nbint=INDEF "
        f"nintfm=INDEF "
        f"plot=no "
        f"norm=0"
    )

    print(f"Running efold in {output_dir} with cmd:\n{cmd}")
    subprocess.run(cmd, shell=True, check=True, cwd=output_dir)

# Helper: Process one set
def process_lc_files(file_list, exposure_ids, instrument_name, periods_df):
    for lc_file, exposure_id in zip(file_list, exposure_ids):
        match = periods_df[periods_df["Exposure_ID"] == exposure_id]

        if match.empty:
            print(f"[WARN] Period not found for {exposure_id} ({instrument_name}). Skipping.")
            continue

        period = match.iloc[0]["Period"]

        output_dir = os.path.join(output_base_folder, exposure_id, instrument_name)
        os.makedirs(output_dir, exist_ok=True)

        output_profile = os.path.join(
            output_dir, f"{exposure_id}_{instrument_name}_pulse_profile_efold_output_same_epoch.fits"
        )

        run_efold(lc_file, period, output_profile, output_dir)


In [36]:

if process_LE:
    print("\nProcessing LE files...")
    process_lc_files(le_files, exposure_ids_le, "LE", periods_le_df)

if process_ME:
    print("\nProcessing ME files...")
    process_lc_files(me_files, exposure_ids_me, "ME", periods_me_df)

if process_HE:
    print("\nProcessing HE files...")
    process_lc_files(he_files, exposure_ids_he, "HE", periods_he_df)


print("\n✅ All efolding done successfully!")


Processing LE files...
Running efold in ./hxmt_efold_output/P050419600101/LE with cmd:
echo '-' | efold 1 /home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196001/pipeline_output/P050419600101_LE_lcnet_2-10keV_0.05s.lc outfile=P050419600101_LE_pulse_profile_efold_output_same_epoch.fits dper=9.79482 nphase=64 epochfo=1 sepoch=20112.43540056619 nbint=INDEF nintfm=INDEF plot=no norm=0
 
efold 1.2 (xronos6.0)
 
 Series 1 file    1:/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_product
 
 Selected FITS extensions: 1 - RATE TABLE;
 
 Source ............ SWIFT_J0243.6+61    Start Time (d) .... 20154 08:59:09.184
 FITS Extension ....  1 - `COUNTS    `   Stop Time (d) ..... 20154 12:16:09.184
 No. of Rows .......        50383        Bin Time (s) ......   0.5000E-01
 Right Ascension ... 40.92               Internal time sys.. Converted to TJD
 Declination ....... 61.43               Experiment ........ HXMT     LE
 
 Corrections applied: Vignetting - No 

100% completed
 
 Intv    1   Start 20154 13:42:43
     Ser.1     Avg  139.2        Chisq  4355.       Var  451.5     Newbs.     64
               Min  105.9          Max  172.0    expVar  6.674      Bins  28062
  Folded light curve ready
Writing output file: P050419600102_LE_pulse_profile_efold_output_same_epoch.fits
Running efold in ./hxmt_efold_output/P050419600201/LE with cmd:
echo '-' | efold 1 /home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_products_pipeline/P0504196/P0504196002/pipeline_output/P050419600201_LE_lcnet_2-10keV_0.05s.lc outfile=P050419600201_LE_pulse_profile_efold_output_same_epoch.fits dper=9.79438 nphase=64 epochfo=1 sepoch=20112.43540056619 nbint=INDEF nintfm=INDEF plot=no norm=0
 
efold 1.2 (xronos6.0)
 
 Series 1 file    1:/home/supremekai/thesis/J0243_6p6124/hxmt/hxmt_me_l3_product
 
 Selected FITS extensions: 1 - RATE TABLE;
 
 Source ............ SWIFT_J0243.6+61    Start Time (d) .... 20155 15:04:28.184
 FITS Extension ....  1 - `COUNTS    `   Stop Tim